In [1]:
# Setup Keys
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import langchain
print(langchain.__version__)

0.3.27


# EssayMaker with Agent Memory

In [3]:
from IPython.display import Image, display
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import AnyMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.types import interrupt, Command
from typing import TypedDict, Annotated, List
from langchain_community.tools.tavily_search import TavilySearchResults
import re
import ast
import json
from pprint import pprint

# Utility function
def extract_dict_from_string(text: str) -> dict:
    """
    Extract and convert a dictionary from a string that may contain extra text.
    Tries JSON first, then falls back to ast.literal_eval.
    """
    # Strip markdown code fences if present
    cleaned = text.strip()

    if re.search(r'```json', cleaned):
      list_substr = cleaned.split("```json")
      cleaned = list_substr[1]
    if re.search(r'```', cleaned):
      list_substr = cleaned.split("```")
      cleaned = list_substr[0]

    cleaned = cleaned.strip()

    try:
        result = json.loads(cleaned)
        if isinstance(result, dict):
            return result
    except json.JSONDecodeError:
        pass

    print(f"❌ Failed to extract dict from response:\n{text[:200]}")
    return {}

# Build tool
search_tool = TavilySearchResults(max_results=2)

# Setup prompt for each task
PROMPT_PLAN = """
  You are an expert essay planner. Your job is to create a detailed and well-structured outline for an essay that will be no longer than 5 paragraphs

  Given the topic and requirements from the user, you must produce an outline that includes:
  1. A strong thesis statement
  2. An introduction section with a hook and context
  3. 3-5 main body sections, each with:
    - A clear section title
    - 2-3 key points to cover
    - A brief description of what each point should address
  4. A conclusion section that ties everything together

  Guidelines:
  - The outline should be logical and flow naturally from one section to the next
  - Each section should build upon the previous one
  - Make sure the outline directly addresses the user's requirements
  - Tailor the depth and complexity to match the required essay length and academic level

  User Input:
  {user_input}

  Agent Output in JSON Format where "" represent the keys :
  - "requirement", derrived from user input
  - "outline", the outline produced by the agent based on the prompt
  - "max_revisions", the maximum revision of the draft essay based on the user input, if it's not specified by the user return 0
  - "max_web_search", the maximum web search that can be done by the agent based on the user input, if it's not specified by the user return 0
"""

PROMPT_RESEARCH = """
  You are an expert research assistant. Your job is to gather relevant information and sources to support an essay based on a given outline.
  If the feedback is not empty, focus on doing search to help gain information to assess the feedback
  Do no more than {max_web_search} search.

  For each section and key point in the outline, you must:
  1. Search the web for relevant, credible sources using tavily_seach tool
  2. Extract key facts, statistics, quotes, or arguments that support the point
  3. Note the source URL and title for citation purposes
  4. Prioritize recent and authoritative sources (academic journals, reputable news, official reports)

  Guidelines:
  - Search for each main section and key point separately to ensure thorough coverage
  - If a search does not return useful results, try rephrasing the query
  - Avoid using low-quality or unreliable sources (forums, opinion blogs without credentials)
  - Collect at least 2-3 supporting pieces of information per key point
  - Organize the collected information according to the outline structure

  Essay Outline:
  {outline}

  Feedback:
  {feedback}

  Agent Output in JSON Format where "" represent the keys :
  - "research", that containsFor each section in the outline, provide with this format
    - Section title
    - Key Point
      - Supporting fact/quote/argument
      - Source: [title] (URL)
"""

PROMPT_GENERATE = """
  You are an expert essay writer. Your job is to write a well-structured, coherent, and polished essay based on the provided outline and research materials.

  Guidelines:
  - Follow the outline structure closely
  - Integrate the research findings naturally into the essay — do not simply list facts
  - Write in a clear, engaging, and appropriate academic tone
  - Each paragraph should have a clear topic sentence and flow logically
  - Use transitions between sections to maintain coherence
  - Support claims with evidence from the research materials
  - Include proper in-text citations where relevant (e.g., "According to [Source], ...")
  - Match the tone and complexity to the user's original requirements
  - Do NOT make up facts or statistics that are not in the provided research

  Essay Outline:
  {outline}

  Research Materials:
  {research}

  User Requirements:
  {requirement}

  Agent Output in JSON Format where "" represent the keys :
  - "essay", Write the full essay from introduction to conclusion.
"""

PROMPT_REFLECT = """
You are an expert essay editor and evaluator. Your job is to critically evaluate an essay and provide detailed, constructive feedback.

Evaluate the essay based on the following criteria:
1. Structure & Organization
   - Does the essay follow a logical flow?
   - Are transitions between sections smooth?
   - Is the outline structure well executed?

2. Thesis & Argumentation
   - Is the thesis clear and strong?
   - Does the essay consistently support the thesis?
   - Are arguments well-developed and convincing?

3. Evidence & Support
   - Is the essay well-supported with evidence?
   - Are sources credible and properly referenced?
   - Are facts and statistics used effectively?

4. Writing Quality
   - Is the tone appropriate and consistent?
   - Is the writing clear and concise?
   - Are there any grammatical or spelling errors?

5. Alignment with Requirements
   - Does the essay meet all of the user's original requirements?
   - Is the length and depth appropriate?
   - Is the target audience addressed correctly?

Essay to Evaluate:
{essay}

User's Original Requirements:
{requirement}

Agent Output in JSON Format where "" represent the keys :
- "score", give the rating from the scale of 0-10, integer only
- "feedback", the detailed evaluation along with the revision priorities
- "need_to_revised", based on the assessment of the agent return only yes/no where yes mean the draft need to be revised, make sure that it's aligned with the 'score'
"""

# Agent state
class AgentState(TypedDict):
    task: str
    requirement: str
    outline: str
    research: str
    essay: str
    feedback: str
    score: int
    need_to_revised: str
    content: List[str]
    revision_number: int

    # user input
    max_revisions: int
    max_web_search: int

# Build model with LangGraph
class EssayWriterAgent:
    def __init__(self, tools, middleware, max_revision:int = 3):
        self.config = {'configurable': {'thread_id': 101}}
        self.max_revision = max_revision
        self.checkpointer = InMemorySaver()

        # Store tools as dictionary
        self.tools = {t.name: t for t in tools}

        # create LLM with tool binding
        self.llm = init_chat_model(
            model="claude-haiku-4-5-20251001",
            temperature=0.5,
        )

        # Bind tools to the model
        self.llm_with_tools = self.llm.bind_tools(tools)

        # to initiate the Graph
        self.graph = StateGraph(AgentState)
        self.graph.add_node('plan', self.call_agent_plan)
        self.graph.add_node('research', self.call_agent_research)
        self.graph.add_edge('plan', 'research')
        self.graph.add_node('generate', self.call_agent_generate)
        self.graph.add_edge('research', 'generate')
        self.graph.add_conditional_edges(
            'generate',
            self.check_revision_iteration,
            {True: 'reflect', False: END})
        self.graph.add_node('reflect', self.call_agent_reflect)
        self.graph.add_edge('reflect', 'research')
        self.graph.set_entry_point('plan')

        self.graph_np = self.graph.compile(
            checkpointer=self.checkpointer,
            )

    def _update_state(self, new_dict:dict, state: AgentState):
      """Check whether the state will be updated based on the JSON output from the LLM response """
      for key, value in new_dict.items():
        if key in state:
          print(f"  ✅ Updated '{key}'")
        else:
          print(f"  ⚠️  Unknown key '{key}' - skipped")

    def _invoke_agent(self, human_message, answer_type:str = 'invoke'):
      """Get the LLM output and add to state"""
      print(f"  ✅ Added HumanMessage to state content")

      # If it's invoke
      if answer_type == 'invoke':

        result = self.llm.invoke([human_message])

        print(f"  ✅ Added Last AIMessage to state content")

        return result

      # If it's stream
      elif answer_type == 'stream':

        messages = [human_message]

        while True:
          response = self.llm_with_tools.invoke(messages)
          messages.append(response)

          # If no tool calls, LLM is done
          if not response.tool_calls:
            pprint(messages[-1])
            print(f"  ✅ Added Last AIMessage to state content")

            return messages[-1]

          # Execute tool calls
          for tc in response.tool_calls:
            print(f"  🔧 Calling tool: {tc['name']}")
            print(f"     Args: {tc['args']}")

            result = self.tools[tc['name']].invoke(tc['args'])

            result_str = str(result)
            print(f"     Result: {result_str[:150]}{'...' if len(result_str) > 150 else ''}\n")

            messages.append(ToolMessage(
                tool_call_id=tc['id'],
                name=tc['name'],
                content=str(result)
            ))

    def call_agent_plan(self, state: AgentState):
      """Function to call agent with the promt to produce the outline of the essay"""
      # Invoke LLM
      message = HumanMessage(content = PROMPT_PLAN.format(
          user_input=state['task']
        )
      )

      print(f"\n{'='*60}")
      print(f"GENERATING OUTLINE")
      print(f"{'='*60}")

      result = self._invoke_agent(message)
      json_result = extract_dict_from_string(result.content)
      self._update_state(json_result, state)

      print(f"\n=== ✅ Requirement ===")
      pprint(f"{json_result.get('requirement', '')}")
      print(f"\n=== ✅ Outline ===")
      pprint(f"{json_result.get('outline', '')}")

      return {
        'outline': json_result.get('outline', ''),
        'requirement': json_result.get('requirement', ''),
        'max_revisions': state['max_revisions'] if json_result.get('max_revisions', 1) == 0 else json_result.get('max_revisions', 1),
        'max_web_search': state['max_web_search'] if json_result.get('max_web_search', 3) == 0 else json_result.get('max_web_search', 3),
        'content': state['content'] + [message, result]
      }

    def call_agent_research(self, state: AgentState):
      """Function to call agent with the prompt to produce the resource to make the essay"""
      # Invoke LLM
      message = HumanMessage(content = PROMPT_RESEARCH.format(
          outline=state['outline'],
          feedback=state['feedback'],
          max_web_search=state['max_web_search']
        )
      )

      print(f"\n{'='*60}")
      print(f"GENERATING RESEARCH")
      print(f"{'='*60}")

      result = self._invoke_agent(message, 'stream')
      json_result = extract_dict_from_string(result.content)
      self._update_state(json_result, state)

      print(f"\n=== ✅ Research ===")
      pprint(f"{json_result.get('research', '')}")

      return {
        'research': json_result.get('research', ''),
        'content': state['content'] + [message, result]
      }

    def call_agent_generate(self, state: AgentState):
      """Function to call agent with the prompt to produce the draft for the essay"""
      # Invoke LLM
      message = HumanMessage(content = PROMPT_GENERATE.format(
          outline=state['outline'],
          research=state['research'],
          requirement=state['requirement'],
        )
      )

      print(f"\n{'='*60}")
      print(f"GENERATING DRAFT - ITERATION {state['revision_number'] + 1}")
      print(f"{'='*60}")

      result = self._invoke_agent(message)
      json_result = extract_dict_from_string(result.content)
      self._update_state(json_result, state)

      print(f"\n=== ✅ Essay ===")
      pprint(f"{json_result.get('essay', '')}")

      return {
        'essay': json_result.get('essay', ''),
        'content': state['content'] + [message, result]
      }

    def check_revision_iteration(self, state: AgentState):
      """Function to decide whether the draft need to be assess by Reflect agent"""

      print(f"\n{'='*60}")
      print(f"DETERMINE DRAFT NEED TO BE ASSESSED ?")
      print(f"{'='*60}")

      # # Draft will always be assessed if it's in the 1st iteration
      if state['revision_number'] < state['max_revisions']:
        return True
      else:
        if state['need_to_revised'] == 'yes' :
          revised_input = input(f"\n⏸️  Reflect agent say it's need to be revised (Score : {state['score']}), would you want to proceed with revision ? (yes/no)").strip().lower()
        elif state['need_to_revised'] != 'yes' :
          revised_input = input(f"\n⏸️  Reflect agent say the draft is already good 👍 (Score : {state['score']}), would you want to proceed with revision ? (yes/no)").strip().lower()

        if revised_input == 'yes' :
          return True
        else :
          return False

    def call_agent_reflect(self, state: AgentState):
      """Function to call agent with the prompt to evaluate the essay draft"""

      # Invoke LLM
      message = HumanMessage(content = PROMPT_REFLECT.format(
          essay=state['essay'],
          requirement=state['requirement'],
        )
      )

      print(f"\n{'='*60}")
      print(f"ASSESSING DRAFT - ITERATION {state['revision_number'] + 1}")
      print(f"{'='*60}")

      result = self._invoke_agent(message)
      json_result = extract_dict_from_string(result.content)
      self._update_state(json_result, state)

      print(f"\n=== ✅ Feedback ===")
      pprint(f"{json_result.get('feedback', '')}")

      print(f"\nDraft Score : {json_result.get('score', 0)}")

      return {
        'feedback': json_result.get('feedback', ''),
        'score': json_result.get('score', 0),
        'need_to_revised': json_result.get('need_to_revised', 'yes'),
        'revision_number': state['revision_number'] + 1,
      }

    def execute(self, message) :
      """Function to run the agent graph flow"""

      self.input_message = message
      print(f"\n{'='*60}")
      print(f"TOPIC FROM THE USER")
      print(f"{'='*60}")
      pprint(f"{self.input_message}")

      # ✅ Initialize ALL required state fields
      initial_state = {
          'task': self.input_message,
          'requirement': '',
          'outline': '',
          'research': '',
          'essay': '',
          'feedback': '',
          'need_to_revised' : 'yes',
          'score': 0,
          'content': [],
          'revision_number': 0,
          'max_revisions': self.max_revision,
          'max_web_search': 3,
      }

      # Run the graph
      result = self.graph_np.invoke(
          initial_state,
          config=self.config
      )

      # Get final answer
      final_answer = result['essay']

      # Change it to HTML mode
      print(f"\n{'='*60}")
      print(f"CHANGE THE ESSAY TO HTML MODE")
      print(f"{'='*60}")

      PROMPT_HTML = """
        Please help change this text to HTML mode, highlight the key point from the essay with bold and red pastel color

        Text : {final_answer}
      """

      message_html = HumanMessage(content = PROMPT_HTML.format(
          final_answer=final_answer,
        )
      )
      final_answer_html = self._invoke_agent(message_html)

      print(f"\n ✅ HTML Generated")

      return final_answer_html


# Define system prompt
bot = EssayWriterAgent([search_tool], [], 3)

# Display LangGraph structure
try:
    # Try to visualize the graph
    display(Image(bot.graph_np.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Visualization error: {e}")
    print("\nGraph structure (ASCII):")
    print(bot.graph_np.get_graph().draw_ascii())

/Users/marcellosusanto/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ImportError: cannot import name 'create_agent' from 'langchain.agents' (/Users/marcellosusanto/Library/Python/3.9/lib/python/site-packages/langchain/agents/__init__.py)

In [ ]:
html = bot.execute("what is the difference between langchain and langsmith, you can do 5 web search and 2 revision")


TOPIC FROM THE USER
('what is the difference between langchain and langsmith, you can do 5 web '
 'search and 2 revision')

GENERATING OUTLINE
  ✅ Added HumanMessage to state content
  ✅ Added Last AIMessage to state content
  ✅ Updated 'requirement'
  ✅ Updated 'max_web_search'
  ✅ Updated 'max_revisions'
  ✅ Updated 'outline'

=== ✅ Requirement ===
('Create a 5-paragraph essay explaining the differences between LangChain and '
 'LangSmith, with support from up to 5 web searches and up to 2 revisions')

=== ✅ Outline ===
("{'thesis_statement': 'While LangChain and LangSmith are complementary tools "
 'within the LLM development ecosystem, they serve distinct purposes: '
 'LangChain is a framework for building applications with language models, '
 'whereas LangSmith is a platform for monitoring, debugging, and optimizing '
 "those applications.', 'introduction': {'hook': 'As large language models "
 'become increasingly central to software development, developers need tools '
 "to bot

# EmailAgent with Agent Memory

In [26]:
from google.colab import auth
from googleapiclient.discovery import build

auth.authenticate_user()
gmail_service = build('gmail', 'v1')

In [28]:
# Get latest data from sheet
creds, _ = default()
gc = gspread.authorize(creds)
sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1Kg-j1_zjdk2UJdVNQanTsF_mLu8S4X9LuXYiBLOBWGA/edit?gid=0#gid=0").sheet1
data = sheet.get_all_records()
df = pd.DataFrame(data)

In [31]:
a = pd.DataFrame()
df = pd.concat([df, a])
df

,name,persona
0,Marcel,Data analyst and scientist who is actively lea...
1,Safrina,Working mom that have store in the supply chai...
2,Peter,Working father that have store in the supply c...


In [ ]:
from IPython.display import Image, display
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import AnyMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.types import interrupt, Command
from typing import TypedDict, Annotated, List
from langchain_community.tools.tavily_search import TavilySearchResults
import re
import ast
import json
from pprint import pprint

import gspread
from google.auth import default
import pandas as pd
import chromadb
from chromadb.config import Settings
import base64
from email.mime.text import MIMEText

# Initialize ChromaDB client
client = chromadb.Client(Settings(
    persist_directory="/content/chroma_db",
    anonymized_telemetry=False
))

# Create or get collection
collection = client.get_or_create_collection(
    name="user_personas",
    metadata={"description": "User persona and preference data"}
)

###########
# UTILITY #
###########
def extract_dict_from_string(text: str) -> dict:
    """
    Extract and convert a dictionary from a string that may contain extra text.
    Tries JSON first, then falls back to ast.literal_eval.
    """
    # Strip markdown code fences if present
    cleaned = text.strip()

    if re.search(r'```json', cleaned):
      list_substr = cleaned.split("```json")
      cleaned = list_substr[1]
    if re.search(r'```', cleaned):
      list_substr = cleaned.split("```")
      cleaned = list_substr[0]

    cleaned = cleaned.strip()

    try:
        result = json.loads(cleaned)
        if isinstance(result, dict):
            return result
    except json.JSONDecodeError:
        pass

    print(f"❌ Failed to extract dict from response:\n{text[:200]}")
    return {}

def sync_personas_from_sheets(url:str, df_append:pd.DataFrame = pd.DataFrame()):
    """Sync persona data from Google Sheets to ChromaDB"""

    # Get latest data from sheet
    creds, _ = default()
    gc = gspread.authorize(creds)
    sheet = gc.open_by_url(url).sheet1
    data = sheet.get_all_records()
    df = pd.DataFrame(data)

    # Append new info
    df = pd.DataFrame

    # Clear existing collection
    client.delete_collection("user_personas")
    collection = client.create_collection("user_personas")

    # Re-populate
    documents = []
    metadatas = []
    ids = []

    for idx, row in df.iterrows():
        doc_text = " | ".join([f"{k}: {v}" for k, v in row.items()])
        documents.append(doc_text)
        metadatas.append({key: str(value) for key, value in row.items()})
        ids.append(f"user_{idx}")

    collection.add(documents=documents, metadatas=metadatas, ids=ids)
    print(f"✅ Synced {len(documents)} personas from Google Sheets")

    return collection

##########
# PROMPT #
##########
BASE_PROMPT = """
  ROLE
  You are an intelligent Email Classification Agent designed to help busy professionals manage their inbox efficiently by categorizing incoming emails based on urgency, relevance, and required action.

  # BACKGROUND: USER PERSONA
  You are assisting {name}, a {persona}
"""

PROMPT_ROUTER = """
  {base_prompt}

  INSTRUCTION
  Classify user intent

  Use "email_workflow" if:
    - User provides an email to analyze/classify
    - User asks to draft/send an email
    - User mentions responding to an email

    Use "general_query" if:
    - User asks about calendar availability
    - User asks to schedule a meeting (without email context)
    - General questions or requests

  INPUT
  {message}

  OUTPUT, in json format
  ```json
  name : [The inferred user name]
  user_intent : [email_workflow/general_query]
  ```
"""

PROMPT_PERSONA = """
  ROLE
  You are an AI that can help inferred the user persona based on the chain of message asked by that user

  INPUT
  {conversation}

  OUTPUT
  Summary of the user persona
"""

PROMPT_TRIAGE = """
  {base_prompt}

  # INSTRUCTION
  For each incoming email, analyze its content, sender, subject, and context to categorize it into one of three actions: IGNORE, NOTIFY, or RESPOND.

  # RULES: EMAIL CATEGORY DEFINITIONS

  ## IGNORE
  Emails that require NO action and NO attention.

  **Criteria:**
  Marketing newsletters, spam emails, mass company announcements

  ## NOTIFY
  Emails that Marcel should be AWARE of but don't require immediate response.

  **Criteria:**
  Team member out sick, build system notifications, project status updates

  ## RESPOND
  Emails that require Marcel's direct action, response, or decision.

  **Criteria:**
  Direct questions from team members, meeting requests, critical bug reports

  INPUT
  Email : {email}

  OUTPUT, in json format
  ```json
  from : [The sender of the email]
  to : [The user that get the email]
  subject : [The subject of the email]
  email_content : [The content of the email]
  email_classification : [IGNORE/NOTIFY/RESPOND]
  classification_reason : [The reason behind the value of email_classification]
  ```
"""

PROMPT_RESPOND = """
  {base_prompt}

  INSTRUCTION
  Draft and call send_email tool with :
  - to_email: recipient's email address
  - subject: email subject line
  - body: the complete email you drafted

  INPUT
  From : {from}
  To : {to}
  Subject : {subject}
  Content : {email_content}

  OUTPUT, in json format
  ```json
  respond_subject : [Subject of the respond email based on user persona]
  respond_content : [Content of the respond email based on user persona]

"""

########
# TOOL #
########
@tool
def send_email(to_email: str, subject: str, body: str) -> str:
  """
  Actually send an email via Gmail API.
  The LLM should provide the to_email, subject, and body.

  Args:
      to_email: Recipient email address
      subject: Email subject line
      body: Complete email body (already drafted by LLM)

  Returns:
      Confirmation message
  """

  try:
      # Create message
      message = MIMEText(body, 'plain')
      message['to'] = to_email
      message['subject'] = subject

      # Encode message
      raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode('utf-8')

      # Send email
      send_message = gmail_service.users().messages().send(
          userId='me',
          body={'raw': raw_message}
      ).execute()

      return f"✅ Email sent successfully to {to_email}! Message ID: {send_message['id']}"

  except Exception as e:
      return f"❌ Failed to send email: {str(e)}"


#########
# AGENT #
#########
class AgentState(TypedDict):
  task: str
  name: str
  persona: str
  user_intent: str
  sender: str
  email_subject: str
  email_content: str
  email_classification: str
  classification_reason: str
  email_respond_subject: str
  email_respond_content: str

class EmailAgent:
  def __init__(self, tools, middleware):
    self.config = {'configurable': {'thread_id': 101}}
    self.checkpointer = InMemorySaver()

    # Store tools as dictionary
    self.tools = {t.name: t for t in tools}

    # create LLM with tool binding
    self.llm = init_chat_model(
        model="claude-haiku-4-5-20251001",
        temperature=0,
    )

    # Bind tools to the model
    self.llm_with_tools = self.llm.bind_tools(tools)

    # to initiate the Graph
    self.graph = StateGraph(AgentState)
    self.graph.add_node('router', self.call_agent_plan)
    self.graph.add_node('persona', self.call_get_user_persona)
    self.graph.add_edge('router', 'persona')
    self.graph.add_node('generate', self.call_agent_generate)
    self.graph.add_edge('research', 'generate')
    self.graph.add_conditional_edges(
        'generate',
        self.check_revision_iteration,
        {True: 'reflect', False: END})
    self.graph.add_node('reflect', self.call_agent_reflect)
    self.graph.add_edge('reflect', 'research')
    self.graph.set_entry_point('plan')

    self.graph_np = self.graph.compile(
        checkpointer=self.checkpointer,
        )

  def _update_state(self, new_dict:dict, state: AgentState):
    """Check whether the state will be updated based on the JSON output from the LLM response """
    for key, value in new_dict.items():
      if key in state:
        print(f"  ✅ Updated '{key}'")
      else:
        print(f"  ⚠️  Unknown key '{key}' - skipped")

  def call_agent_router(self, state:AgentState):

    # Initial sync of the user persona data
    sync_personas_from_sheets("https://docs.google.com/spreadsheets/d/1Kg-j1_zjdk2UJdVNQanTsF_mLu8S4X9LuXYiBLOBWGA/edit?gid=0#gid=0")

    # Invoke LLM
    message = HumanMessage(content = PROMPT_ROUTER.format(
        base_prompt=BASE_PROMPT,
        message=state['task'],
      )
    )

    print(f"\n{'='*60}")
    print(f"ROUTING}")
    print(f"{'='*60}")

    result = self.llm.invoke(message)
    json_result = extract_dict_from_string(result.content)
    self._update_state(json_result, state)

    print(f"\nUser : {json_result.get('name', '')}")
    print(f"\Task Type : {json_result.get('user_intent', '')}")

    return {
      'name': json_result.get('name', ''),
      'user_intent': json_result.get('user_intent', ''),
    }

  def call_get_user_persona(self, state:AgentState):
    """ Retrieve user persona and preference information """

    # Filter by specific user
    result = collection.get(
          where={"name": state['name']}
      )

    if len(result['metadata']) > 0 :
      print(f"✅ User Persona {state['name']} found, proceed with it")

      return result['metadata'][0]

    else:
      if state['']
      print(f"❌ No user persona found, proceed with default")
      result = {
          'name':state['name'],
          'persona':'User not specified, treat it as default persona'
      }

  def execute(self, message) :
    """Function to run the agent graph flow"""

    self.input_message = message
    print(f"\n{'='*60}")
    print(f"TOPIC FROM THE USER")
    print(f"{'='*60}")
    pprint(f"{self.input_message}")

    # ✅ Initialize ALL required state fields
    initial_state = {
        'task': self.input_message,
        'name': '',
        'persona': '',
        'user_intent': '',
        'sender': '',
        'email_subject': '',
        'email_content' : '',
        'email_classification': '',
        'classification_reason': '',
        'email_respond_subject': '',
        'email_respond_content' : ''
    }

    # Run the graph
    result = self.graph_np.invoke(
        initial_state,
        config=self.config
    )

    # Get final answer
    final_answer = result['essay']



ModuleNotFoundError: No module named 'prompts'